In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from optdesign import opt_dmetric, opt_maxvol, setup_greedy, setup_random

# Three Gaussians

In [ ]:
config = np.array([[0.1, 0.2], [0.2, 0.1], [0.6, 0.5]])
npar = len(config)


def compute_basis(xs):
    return np.array([np.exp(-0.5 * ((xs - mu) / width) ** 2) for mu, width in config])


npt = 100
sigma = 0.01
pars0 = np.array([0.3, 0.5, 0.1])
rng = np.random.default_rng(42)
xs = rng.uniform(0, 1, npt)
basisfns = compute_basis(xs)
design_matrix = basisfns.T / sigma
expected_values = (np.dot(basisfns.T, pars0) + rng.normal(0, sigma, npt)) / sigma


def linear_regression(irows=None):
    if irows is None:
        irows = range(npt)
    u, s, vt = np.linalg.svd(design_matrix[irows], full_matrices=False)
    pars = np.dot(vt.T, np.dot(u.T, expected_values[irows]) / s)
    sigma_evs = 1 / s
    sigma_pars = np.sqrt(np.einsum("ji,j,ji->i", vt, 1 / s**2, vt))
    settings = {"precision": 3, "suppress_small": True, "separator": "  "}
    print("pars      ", np.array2string(pars, **settings))
    print("sigma_evs ", np.array2string(sigma_evs, **settings))
    print("sigma_pars", np.array2string(sigma_pars, **settings))
    return pars


def plot_gauss(pars=None, irows=None):
    xgrid = np.linspace(0, 1, npt)
    fig, ax = plt.subplots(dpi=100)
    basisfns_grid = compute_basis(xgrid)
    for ib, bgrid in enumerate(basisfns_grid):
        ax.plot(xgrid, bgrid, ls=":", label=f"basis-{ib}", color="gray")
    ax.plot(xs, expected_values * sigma, marker="o", color="#BBBBBB", ls="None")
    if irows is not None:
        ax.plot(xs[irows], expected_values[irows] * sigma, "k+")
        ax.set_title(f"{len(irows)} points selected")
    else:
        ax.set_title("No points selected")
    ax.plot(xgrid, np.dot(pars0, basisfns_grid), label="exact")
    if pars is not None:
        ax.plot(xgrid, np.dot(pars, basisfns_grid), label="fit")
    ax.legend(loc=0)
    fig.tight_layout()
    plt.show(fig)
    plt.close(fig)


plot_gauss()

### Using all points

In [ ]:
pars = linear_regression()
plot_gauss(pars, range(npt))

### Random setup

In [ ]:
irows = setup_random(design_matrix)[:npar]
pars = linear_regression(irows)
plot_gauss(pars, irows)

### Greedy setup

In [ ]:
irows = setup_greedy(design_matrix)[:npar]
pars = linear_regression(irows)
plot_gauss(pars, irows)

### Maximum Volume selection

In [ ]:
irows = opt_maxvol(design_matrix)[:npar]
pars = linear_regression(irows)
plot_gauss(pars, irows)

### D-metric selection

In [ ]:
nrow = 9
irows = opt_dmetric(design_matrix, nrow)[:nrow]
pars = linear_regression(irows)
plot_gauss(pars, irows)